In [1]:
from __future__ import print_function

from keras.models import Sequential, Model
from keras.layers.embeddings import Embedding
from keras.layers import Input, Activation, Dense, Permute, Dropout, add, dot, concatenate
from keras.layers import LSTM
from keras.utils.data_utils import get_file
from keras.preprocessing.sequence import pad_sequences
from functools import reduce
import tarfile
import numpy as np
import re

from nltk.tokenize import word_tokenize

Using TensorFlow backend.


In [2]:
def tokenize(sent):
    '''Return the tokens of a sentence including punctuation.
    >>> tokenize('Bob dropped the apple. Where is the apple?')
    ['Bob', 'dropped', 'the', 'apple', '.', 'Where', 'is', 'the', 'apple', '?']
    '''
    return [x.strip() for x in re.split('(\W+)?', sent) if x.strip()]

In [3]:
word_tokenize('Bob dropped the apple. Where is the apple?')

['Bob', 'dropped', 'the', 'apple', '.', 'Where', 'is', 'the', 'apple', '?']

In [4]:
def parse_stories(lines, only_supporting=False):
    '''Parse stories provided in the bAbi tasks format
    If only_supporting is true, only the sentences
    that support the answer are kept.
    '''
    data = []
    story = []
    for line in lines:
        line = line.decode('utf-8').strip()
        nid, line = line.split(' ', 1)
        #print(nid)
        #print(line)
        nid = int(nid)
        if nid == 1:
            story = []
        if '\t' in line:
            q, a, supporting = line.split('\t')
            #print(q, a , supporting)
            q = tokenize(q)
            substory = None
            if only_supporting:
                # Only select the related substory
                supporting = map(int, supporting.split())
                substory = [story[i - 1] for i in supporting]
                #print(substory)
            else:
                # Provide all the substories
                substory = [x for x in story if x]
                #print(substory)
            data.append((substory, q, a))
            story.append('')
        else:
            sent = tokenize(line)
            story.append(sent)
    return data

In [5]:
def get_stories(f, only_supporting=False, max_length=None):
    '''Given a file name, read the file,
    retrieve the stories,
    and then convert the sentences into a single story.
    If max_length is supplied,
    any stories longer than max_length tokens will be discarded.
    '''
    data = parse_stories(f.readlines(), only_supporting=only_supporting)
    flatten = lambda data: reduce(lambda x, y: x + y, data)
    data = [(flatten(story), q, answer) for story, q, answer in data if not max_length or len(flatten(story)) < max_length]
    return data

In [6]:
def vectorize_stories(data, word_idx, story_maxlen, query_maxlen):
    X = []
    Xq = []
    Y = []
    for story, query, answer in data:
        x = [word_idx[w] for w in story]
        xq = [word_idx[w] for w in query]
        # let's not forget that index 0 is reserved
        y = np.zeros(len(word_idx) + 1)
        y[word_idx[answer]] = 1
        X.append(x)
        Xq.append(xq)
        Y.append(y)
    return (pad_sequences(X, maxlen=story_maxlen),
            pad_sequences(Xq, maxlen=query_maxlen), np.array(Y))

In [7]:
try:
    path = get_file('babi-tasks-v1-2.tar.gz', origin='https://s3.amazonaws.com/text-datasets/babi_tasks_1-20_v1-2.tar.gz')
except:
    print('Error downloading dataset, please download it manually:\n'
          '$ wget http://www.thespermwhale.com/jaseweston/babi/tasks_1-20_v1-2.tar.gz\n'
          '$ mv tasks_1-20_v1-2.tar.gz ~/.keras/datasets/babi-tasks-v1-2.tar.gz')
    raise
tar = tarfile.open(path)

In [8]:
challenges = {
    # QA1 with 10,000 samples
    'single_supporting_fact_10k': 'tasks_1-20_v1-2/en-10k/qa1_single-supporting-fact_{}.txt',
    # QA2 with 10,000 samples
    'two_supporting_facts_10k': 'tasks_1-20_v1-2/en-10k/qa2_two-supporting-facts_{}.txt',
}

challenge_type = 'single_supporting_fact_10k'
challenge = challenges[challenge_type]

In [9]:
tar.extractfile(challenge.format('train'))

<ExFileObject name='/Users/madhavthaker/.keras/datasets/babi-tasks-v1-2.tar.gz'>

In [10]:
train_stories = get_stories(tar.extractfile(challenge.format('train')))
test_stories = get_stories(tar.extractfile(challenge.format('test')))

/anaconda3/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


In [26]:
vocab = set()
for story, q, answer in train_stories + test_stories:
    vocab |= set(story + q + [answer])
vocab = sorted(vocab)

In [45]:
print (vocab)

['.', '?', 'Daniel', 'John', 'Mary', 'Sandra', 'Where', 'back', 'bathroom', 'bedroom', 'garden', 'hallway', 'is', 'journeyed', 'kitchen', 'moved', 'office', 'the', 'to', 'travelled', 'went']


In [12]:
# Reserve 0 for masking via pad_sequences
vocab_size = len(vocab) + 1
story_maxlen = max(map(len, (x for x, _, _ in train_stories + test_stories)))
query_maxlen = max(map(len, (x for _, x, _ in train_stories + test_stories)))

In [13]:
print(vocab_size, story_maxlen, query_maxlen)

22 68 4


In [14]:
# Reserve 0 for masking via pad_sequences
vocab_size = len(vocab) + 1
story_maxlen = max(map(len, (x for x, _, _ in train_stories + test_stories)))
query_maxlen = max(map(len, (x for _, x, _ in train_stories + test_stories)))

In [15]:
word_idx = dict((c, i + 1) for i, c in enumerate(vocab))
inputs_train, queries_train, answers_train = vectorize_stories(train_stories,
                                                               word_idx,
                                                               story_maxlen,
                                                               query_maxlen)
inputs_test, queries_test, answers_test = vectorize_stories(test_stories,
                                                            word_idx,
                                                            story_maxlen,
                                                            query_maxlen)

In [16]:
print('-')
print('inputs: integer tensor of shape (samples, max_length)')
print('inputs_train shape:', inputs_train.shape)
print('inputs_test shape:', inputs_test.shape)
print('-')
print('queries: integer tensor of shape (samples, max_length)')
print('queries_train shape:', queries_train.shape)
print('queries_test shape:', queries_test.shape)
print('-')
print('answers: binary (1 or 0) tensor of shape (samples, vocab_size)')
print('answers_train shape:', answers_train.shape)
print('answers_test shape:', answers_test.shape)
print('-')
print('Compiling...')

-
inputs: integer tensor of shape (samples, max_length)
inputs_train shape: (10000, 68)
inputs_test shape: (1000, 68)
-
queries: integer tensor of shape (samples, max_length)
queries_train shape: (10000, 4)
queries_test shape: (1000, 4)
-
answers: binary (1 or 0) tensor of shape (samples, vocab_size)
answers_train shape: (10000, 22)
answers_test shape: (1000, 22)
-
Compiling...


In [17]:
# placeholders
input_sequence = Input((story_maxlen,))
question = Input((query_maxlen,))

# encoders
# embed the input sequence into a sequence of vectors
input_encoder_m = Sequential()
input_encoder_m.add(Embedding(input_dim=vocab_size,
                              output_dim=64))
input_encoder_m.add(Dropout(0.3))
# output: (samples, story_maxlen, embedding_dim)

# embed the input into a sequence of vectors of size query_maxlen
input_encoder_c = Sequential()
input_encoder_c.add(Embedding(input_dim=vocab_size,
                              output_dim=query_maxlen))
input_encoder_c.add(Dropout(0.3))
# output: (samples, story_maxlen, query_maxlen)

# embed the question into a sequence of vectors
question_encoder = Sequential()
question_encoder.add(Embedding(input_dim=vocab_size,
                               output_dim=64,
                               input_length=query_maxlen))
question_encoder.add(Dropout(0.3))
# output: (samples, query_maxlen, embedding_dim)

# encode input sequence and questions (which are indices)
# to sequences of dense vectors
input_encoded_m = input_encoder_m(input_sequence)
input_encoded_c = input_encoder_c(input_sequence)
question_encoded = question_encoder(question)

In [18]:

# compute a 'match' between the first input vector sequence
# and the question vector sequence
# shape: `(samples, story_maxlen, query_maxlen)`
match = dot([input_encoded_m, question_encoded], axes=(2, 2))
match = Activation('softmax')(match)

# add the match matrix with the second input vector sequence
response = add([match, input_encoded_c])  # (samples, story_maxlen, query_maxlen)
response = Permute((2, 1))(response)  # (samples, query_maxlen, story_maxlen)

# concatenate the match matrix with the question vector sequence
answer = concatenate([response, question_encoded])

# the original paper uses a matrix multiplication for this reduction step.
# we choose to use a RNN instead.
answer = LSTM(32)(answer)  # (samples, 32)

# one regularization layer -- more would probably be needed.
answer = Dropout(0.3)(answer)
answer = Dense(vocab_size)(answer)  # (samples, vocab_size)
# we output a probability distribution over the vocabulary
answer = Activation('softmax')(answer)

# build the final model
model = Model([input_sequence, question], answer)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])

# train
model.fit([inputs_train, queries_train], answers_train,
          batch_size=32,
          epochs=120,
          validation_data=([inputs_test, queries_test], answers_test))

Train on 10000 samples, validate on 1000 samples
Epoch 1/120
10000/10000 [==============================] - 6s 631us/step - loss: 1.9487 - acc: 0.1715 - val_loss: 1.7802 - val_acc: 0.2670
Epoch 2/120
10000/10000 [==============================] - 6s 626us/step - loss: 1.7335 - acc: 0.2321 - val_loss: 1.6798 - val_acc: 0.2620
Epoch 3/120
10000/10000 [==============================] - 7s 682us/step - loss: 1.6703 - acc: 0.2672 - val_loss: 1.6370 - val_acc: 0.3300
Epoch 4/120
10000/10000 [==============================] - 7s 681us/step - loss: 1.5926 - acc: 0.3421 - val_loss: 1.5124 - val_acc: 0.4070
Epoch 5/120
10000/10000 [==============================] - 6s 602us/step - loss: 1.5184 - acc: 0.3988 - val_loss: 1.4818 - val_acc: 0.4210
Epoch 6/120
10000/10000 [==============================] - 6s 590us/step - loss: 1.5052 - acc: 0.4051 - val_loss: 1.4603 - val_acc: 0.4400
Epoch 7/120
10000/10000 [==============================] - 6s 583us/step - loss: 1.4735 - acc: 0.4334 - val_loss: 1.4

10000/10000 [==============================] - 6s 643us/step - loss: 0.3310 - acc: 0.8795 - val_loss: 0.3922 - val_acc: 0.8590
Epoch 60/120
10000/10000 [==============================] - 7s 668us/step - loss: 0.3221 - acc: 0.8831 - val_loss: 0.3522 - val_acc: 0.8720
Epoch 61/120
10000/10000 [==============================] - 7s 738us/step - loss: 0.3155 - acc: 0.8850 - val_loss: 0.3538 - val_acc: 0.8630
Epoch 62/120
10000/10000 [==============================] - 7s 742us/step - loss: 0.2951 - acc: 0.8898 - val_loss: 0.3392 - val_acc: 0.8690
Epoch 63/120
10000/10000 [==============================] - 7s 725us/step - loss: 0.2999 - acc: 0.8919 - val_loss: 0.3281 - val_acc: 0.8790
Epoch 64/120
10000/10000 [==============================] - 7s 709us/step - loss: 0.2903 - acc: 0.8928 - val_loss: 0.3192 - val_acc: 0.8830
Epoch 65/120
10000/10000 [==============================] - 7s 739us/step - loss: 0.2722 - acc: 0.9012 - val_loss: 0.3088 - val_acc: 0.8920
Epoch 66/120
10000/10000 [=======

10000/10000 [==============================] - 7s 678us/step - loss: 0.0727 - acc: 0.9763 - val_loss: 0.1733 - val_acc: 0.9460
Epoch 118/120
10000/10000 [==============================] - 7s 658us/step - loss: 0.0680 - acc: 0.9770 - val_loss: 0.1455 - val_acc: 0.9580
Epoch 119/120
10000/10000 [==============================] - 7s 696us/step - loss: 0.0677 - acc: 0.9779 - val_loss: 0.1513 - val_acc: 0.9520
Epoch 120/120
10000/10000 [==============================] - 7s 692us/step - loss: 0.0627 - acc: 0.9794 - val_loss: 0.1601 - val_acc: 0.9540


Let's test to see how the model performs.

We are going to predict the answer to the first question within the testing data set. The conversation and question is as follows:

`1 John travelled to the hallway.
2 Mary journeyed to the bathroom.
3 Where is John? 	hallway	1`

In [51]:
print(model.predict([inputs_test, queries_test])[0].argmax())
print(answers_test[0].argmax())

12
12


We see that they both provide the indentical answer and to translate to actual text, we first subtract 1 since index zero is reserved. So let us take a look at the 11th index within our vocab list.

In [52]:
print(vocab[11])

hallway


What do you know, the model was able to understand and answer the question properly.